In [1]:
import pandas as pd
from datetime import datetime

excel_obj = pd.ExcelFile('mock_isp_data.xlsx')
sheets_name = excel_obj.sheet_names
print(sheets_name)

df = excel_obj.parse('Main_Data')

df['MSISDN'] = df['MSISDN'].apply(str)

date_cols = [
    'CONTRACT_START_DT',
    'CONTRACT_END_DT'
]

df[date_cols] = df[date_cols].apply(pd.to_datetime)

df.info()
df



['Main_Data', 'Metadata']
<class 'pandas.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   MSISDN             5 non-null      str           
 1   RC_RATE            5 non-null      int64         
 2   SUBS_STATUS        5 non-null      str           
 3   CUST_FULL_NAME     5 non-null      str           
 4   PRODUCT_TYPE       5 non-null      str           
 5   CONTRACT_START_DT  5 non-null      datetime64[us]
 6   CONTRACT_END_DT    5 non-null      datetime64[us]
dtypes: datetime64[us](2), int64(1), str(4)
memory usage: 412.0 bytes


,MSISDN,RC_RATE,SUBS_STATUS,CUST_FULL_NAME,PRODUCT_TYPE,CONTRACT_START_DT,CONTRACT_END_DT
0,960673105,599,Active,ABC Corp,FTTH,2025-03-01,2026-03-01
1,912345678,899,Active,"XYZ Co.,Ltd",Mobile,2025-05-15,2026-05-15
2,812345678,1200,Suspended,Pah Nattapong Service,Mobile,2024-10-20,2025-10-20
3,612345678,449,Active,JukThong Tech,DATA SERVICE,2024-01-10,2025-01-10
4,998887776,299,Terminated,Sample Client,FTTH,2024-12-01,2025-12-01


In [2]:
today = pd.Timestamp(datetime.now().date())

# คำนวณวันคงเหลือสัญญา
df['REMAIN_DAYS'] = (df['CONTRACT_END_DT'] - today).dt.days

# มูลค่าสัญญา
df['REMAIN_VALUE'] = df['REMAIN_DAYS'] * (df['RC_RATE'] / 30)


df[['CUST_FULL_NAME', 'REMAIN_DAYS', 'REMAIN_VALUE']]

,CUST_FULL_NAME,REMAIN_DAYS,REMAIN_VALUE
0,ABC Corp,12,239.600000
1,"XYZ Co.,Ltd",87,2607.100000
2,Pah Nattapong Service,-120,-4800.000000
3,JukThong Tech,-403,-6031.566667
4,Sample Client,-78,-777.400000


In [3]:
expired_list = df[df['REMAIN_DAYS'] < 0]
print(f'สัญญาที่หมดอายุแล้ว {len(expired_list)} ราย')
expired_list

สัญญาที่หมดอายุแล้ว 3 ราย


,MSISDN,RC_RATE,SUBS_STATUS,CUST_FULL_NAME,PRODUCT_TYPE,CONTRACT_START_DT,CONTRACT_END_DT,REMAIN_DAYS,REMAIN_VALUE
2,812345678,1200,Suspended,Pah Nattapong Service,Mobile,2024-10-20,2025-10-20,-120,-4800.000000
3,612345678,449,Active,JukThong Tech,DATA SERVICE,2024-01-10,2025-01-10,-403,-6031.566667
4,998887776,299,Terminated,Sample Client,FTTH,2024-12-01,2025-12-01,-78,-777.400000


In [4]:
warning_list = df[(df['REMAIN_DAYS'] >= 0) & (df['REMAIN_DAYS'] < 30) ]

print(f'สาขาที่ไกล้หมดสัญญาแล้ว {len(warning_list)} รายการ')
warning_list[['CUST_FULL_NAME', 'MSISDN','REMAIN_DAYS']]

สาขาที่ไกล้หมดสัญญาแล้ว 1 รายการ


,CUST_FULL_NAME,MSISDN,REMAIN_DAYS
0,ABC Corp,960673105,12


In [5]:
status_active = df[df['SUBS_STATUS'] == 'Active']
mini_show = ['CUST_FULL_NAME', 'MSISDN','REMAIN_DAYS']
print(f'สาขาที่ยังใช้บริการอยู่ {len(status_active)} สาขา')
status_active[mini_show]

สาขาที่ยังใช้บริการอยู่ 3 สาขา


,CUST_FULL_NAME,MSISDN,REMAIN_DAYS
0,ABC Corp,960673105,12
1,"XYZ Co.,Ltd",912345678,87
3,JukThong Tech,612345678,-403
